In [1]:
#general
import io
import os
import shutil
import requests
import base64

#pandas
import pandas as pd

#numpy
import numpy as np

#matplotlib.pylot
import matplotlib.pyplot as plt

#sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense, Activation

Using TensorFlow backend.


In [2]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import requests
import base64


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [3]:
#path
path = "/Users/petronillagriffith/desktop/Fifa"

#import file
filename = os.path.join(path, "fifa_ranking.csv")

#names each column
names = ['rank', 'country', 'total_points', 'previous_points', 'rank_change', 'cur_year_avg', 
          'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted', 'two_year_ago_avg', 
          'two_year_ago_avg_weighted', 'three_year_ago_avg_', 'three_year_ago_avg_weighted', 'confederation', 'rank_date']

#making a DataFrame
df = pd.read_csv(filename, encoding = "ISO-8859-1", sep=',', na_values=['-1'], index_col=False)

#writes out "Starting File:"
print("Starting File:")


print(df[20:100])


print()

#writes out "Ending File:"
print("Ending File:")

print(df[-10:1])

Starting File:
    rank         country_full country_abrv  total_points  previous_points  \
20    21              Romania          ROU           0.0               51   
21    22              Uruguay          URU           0.0               47   
22    23             Portugal          POR           0.0               38   
23    24             Cameroon          CMR           0.0               43   
24    25               Zambia          ZAM           0.0               38   
25    26                Egypt          EGY           0.0               44   
26    27                Wales          WAL           0.0               40   
27    28                  USA          USA           0.0               43   
28    29       CÃ´te d'Ivoire          CIV           0.0               41   
29    30             Scotland          SCO           0.0               43   
30    31              Tunisia          TUN           0.0               35   
31    32               Greece          GRE           0.0     

In [70]:
df.head(5)

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,country_full_class,country_abrv_class,confederation_class,rank_date_class
0,1,0.0,57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
1,2,0.0,57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0,0
2,3,0.0,50,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2,0,0
3,4,0.0,55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3,0,0
4,5,0.0,51,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4,1,0


In [6]:
a = df['country_full'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['country_full_class'] = df['country_full'].map(lambda x: byte_offset_map[x])

In [7]:
a = df['country_abrv'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['country_abrv_class'] = df['country_abrv'].map(lambda x: byte_offset_map[x])

In [8]:
a = df['confederation'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['confederation_class'] = df['confederation'].map(lambda x: byte_offset_map[x])

In [9]:
a = df['rank_date'].unique().tolist()

byte_offset_map = {}

for i, id in enumerate(a): byte_offset_map[id] = i
    
df['rank_date_class'] = df['rank_date'].map(lambda x: byte_offset_map[x])

In [10]:
df[1000:1003]

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date,country_full_class,country_abrv_class,confederation_class,rank_date_class
1000,164,Aruba,ARU,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,1994-02-15,163,163,2,5
1001,165,Chad,CHA,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-02-15,161,161,3,5
1002,165,St Kitts and Nevis,SKN,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,1994-02-15,168,168,2,5


In [11]:
#Drops the 'country' column from the data set
df = df.drop(['country_full'], axis=1)

In [12]:
#Drops the 'country' column from the data set
df = df.drop(['country_abrv'], axis=1)

In [13]:
#Drops the 'confederation' column from the data set
df = df.drop(['confederation'], axis=1)

In [14]:
#Drops the 'confederation' column from the data set
df = df.drop(['rank_date'], axis=1)

In [15]:
df[1000:1003]

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,country_full_class,country_abrv_class,confederation_class,rank_date_class
1000,164,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,163,163,2,5
1001,165,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161,161,3,5
1002,165,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168,168,2,5


In [54]:
df.dtypes

rank                         int64
total_points               float64
previous_points              int64
rank_change                float64
cur_year_avg               float64
cur_year_avg_weighted      float64
last_year_avg              float64
last_year_avg_weighted     float64
two_year_ago_avg           float64
two_year_ago_weighted      float64
three_year_ago_avg         float64
three_year_ago_weighted    float64
country_full_class           int64
country_abrv_class           int64
confederation_class          int64
rank_date_class              int64
dtype: object

In [33]:
max = df.loc[df['country_full_class'].idxmax()]

In [49]:
total_country = max['country_full_class']

In [50]:
total_country

215.0